# 初期設定

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import time
from selenium.webdriver.support.ui import Select
import pandas as pd
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import re

In [2]:
# ブラウザをheadlessモード実行
options = webdriver.ChromeOptions()
#ヘッドレスモード（バックグラウンドで起動）で実行。コラボの場合、必須。
options.add_argument('--headless')
#サンドボックスモードの解除。これも必須。
options.add_argument('--no-sandbox')
#これも設定した方がよい。
options.add_argument('--disable-dev-shm-usage')
wait = WebDriverWait(options,10)

In [3]:
driver = webdriver.Chrome()
#指定したドライバーが見つかるまで待機
driver.implicitly_wait(10)

#urlの指定
url="https://www.google.co.jp/"

driver.get(url)
time.sleep(3)

print("サイトのタイトル：", driver.title)

サイトのタイトル： Google


In [4]:
driver = webdriver.Chrome()
#指定したドライバーが見つかるまで待機
driver.implicitly_wait(10)

#urlの指定
url="https://www.data.jma.go.jp/stats/etrn/view/daily_s1.php?prec_no=44&block_no=47662&year=2023&month=12&day=&view="

driver.get(url)
time.sleep(3)

day = []
kiatu_list = []
kionn_list = []
situdo_list = []

for date in range(5,36):
    a = driver.find_element(By.XPATH, f'//*[@id="tablefix1"]/tbody/tr[{date}]/td[1]/div/a').text
    day.append(a)

    kiatu = driver.find_element(By.XPATH, f'//*[@id="tablefix1"]/tbody/tr[{date}]/td[2]').text
    kiatu_list.append(kiatu)

    kionn = driver.find_element(By.XPATH, f'//*[@id="tablefix1"]/tbody/tr[{date}]/td[7]').text
    kionn_list.append(kionn)
    
    situdo = driver.find_element(By.XPATH, f'//*[@id="tablefix1"]/tbody/tr[{date}]/td[10]').text
    situdo_list.append(situdo)
print(day)
print(kiatu_list)
print(kionn_list)
print(situdo_list)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
['1012.3', '1015.6', '1015.5', '1018.2', '1016.3', '1008.8', '1004.1', '1016.3', '1017.0', '1018.5', '1019.6', '1007.6', '1021.2', '1024.3', '1011.8', '1001.4', '1007.5', '1020.5', '1020.2', '1008.7', '1008.3', '1015.0', '1023.8', '1023.9', '1018.2', '1019.2', '1022.5', '1022.6', '1018.2', '1017.3', '1004.0']
['8.7', '8.3', '9.1', '8.7', '8.1', '11.4', '12.9', '12.3', '11.9', '13.0', '13.4', '11.9', '10.8', '10.2', '11.0', '17.0', '11.2', '7.0', '7.0', '8.5', '7.7', '5.3', '5.0', '4.9', '6.3', '7.6', '8.4', '7.0', '7.8', '9.0', '9.2']
['54', '60', '58', '53', '64', '77', '60', '49', '66', '65', '74', '92', '52', '62', '82', '71', '36', '36', '53', '59', '41', '38', '47', '55', '56', '58', '50', '56', '61', '63', '79']


In [5]:
import sqlite3

# データベースに接続または作成
conn = sqlite3.connect('tenki.sqlite')
cursor = conn.cursor()

In [6]:
# テーブルの作成
cursor.execute('''
    CREATE TABLE IF NOT EXISTS weather_data (
        id INTEGER PRIMARY KEY,
        date TEXT,
        pressure TEXT,
        temperature TEXT,
        humidity TEXT
    )
''')

In [7]:
# データベースに挿入
for i in range(len(day)):
    cursor.execute('''
        INSERT INTO weather_data (date, pressure, temperature, humidity) VALUES (?, ?, ?, ?)
    ''', (day[i], kiatu_list[i], kionn_list[i], situdo_list[i]))

# コミットとクローズ
conn.commit()
conn.close()

In [8]:
#テーブルの削除
# SQLiteデータベースに接続
conn = sqlite3.connect('tenki.sqlite')
cursor = conn.cursor()

# テーブルを削除するSQL文の実行
cursor.execute('DROP TABLE IF EXISTS local_tenki;')

# 変更をコミット
conn.commit()

# 接続を閉じる
conn.close()

In [9]:
import csv

# SQLiteデータベースに接続
conn = sqlite3.connect('tenki.sqlite')
cursor = conn.cursor()

# テーブルの作成local
cursor.execute('''
    CREATE TABLE IF NOT EXISTS local_tenki (
        date TEXT PRIMARY KEY,
        steps TEXT,
        stride_median TEXT,
        calories_burned TEXT,
        sedentary_calories_burned TEXT
    )
''')

# コミットとクローズ
conn.commit()
conn.close()

In [10]:
import csv

# SQLiteデータベースに接続
conn = sqlite3.connect('tenki.sqlite')
cursor = conn.cursor()

# CSVファイルの読み込みとデータベースへの挿入
with open('/Users/takuto/dspro_last/ds_tenki/dspro_ローカルデータ.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)

    # ヘッダー行がある場合、読み飛ばす
    next(csv_reader)

    for row in csv_reader:
        cursor.execute('''
            INSERT INTO local_tenki (date, steps, stride_median, calories_burned, sedentary_calories_burned)
            VALUES (?, ?, ?, ?, ?)
        ''', (row[0], row[1], row[2], row[3], row[4]))

# コミットとクローズ
conn.commit()
conn.close()

# 2322008会長に教えてもらった